In [51]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as fn

spark = SparkSession \
    .builder \
    .appName("ejercicios") \
    .getOrCreate()

In [52]:
df_votes = spark.read.csv('PECElecciones.csv', sep=";", header=True)

In [53]:
parties = df_votes.columns
parties

['Codigo',
 'Mesas',
 'Censo',
 'Votantes',
 'Validos',
 'Blanco',
 'Nulos',
 'PP',
 'PSOE',
 'PODEMOS_IU_EQUO',
 'Cs',
 'ECP',
 'PODEMOS_COMPROMIS_EUPV',
 'ERC_CATS',
 'CDC',
 'PODEMOS_EN_MAREA_ANOVA_EU',
 'EAJ_PNV',
 'EH_Bildu',
 'CCa_PNC',
 'PACMA',
 'RECORTES_CERO_GRUPO_VERDE',
 'UPyD',
 'VOX',
 'BNG_NS',
 'PCPE',
 'GBAI',
 'EB',
 'FE_de_las_JONS',
 'SI',
 'SOMVAL',
 'CCD',
 'SAIn',
 'PH',
 'CENTRO_MODERADO',
 'P_LIB',
 'CCD_CI',
 'UPL',
 'PCOE',
 'AND',
 'JXC',
 'PFyV',
 'CILUS',
 'PxC',
 'MAS',
 'IZAR',
 'UNIDAD_DEL_PUEBLO',
 'PREPAL',
 'Ln',
 'REPO',
 'INDEPENDIENTES_FIA',
 'ENTABAN',
 'IMC',
 'PUEDE',
 'FE',
 'ALCD',
 'FME',
 'HRTS_Ln',
 'UDT']

In [54]:
parties.remove('Codigo')
parties.remove('Mesas')
parties.remove('Censo')
parties.remove('Votantes')
parties.remove('Validos')



In [55]:
len(parties)

53

In [56]:
array_of_cols = ["'{0}', {0}".format(p) for p in parties]
print(array_of_cols)

string_of_cols = ", ".join(array_of_cols)
print(string_of_cols)

["'Blanco', Blanco", "'Nulos', Nulos", "'PP', PP", "'PSOE', PSOE", "'PODEMOS_IU_EQUO', PODEMOS_IU_EQUO", "'Cs', Cs", "'ECP', ECP", "'PODEMOS_COMPROMIS_EUPV', PODEMOS_COMPROMIS_EUPV", "'ERC_CATS', ERC_CATS", "'CDC', CDC", "'PODEMOS_EN_MAREA_ANOVA_EU', PODEMOS_EN_MAREA_ANOVA_EU", "'EAJ_PNV', EAJ_PNV", "'EH_Bildu', EH_Bildu", "'CCa_PNC', CCa_PNC", "'PACMA', PACMA", "'RECORTES_CERO_GRUPO_VERDE', RECORTES_CERO_GRUPO_VERDE", "'UPyD', UPyD", "'VOX', VOX", "'BNG_NS', BNG_NS", "'PCPE', PCPE", "'GBAI', GBAI", "'EB', EB", "'FE_de_las_JONS', FE_de_las_JONS", "'SI', SI", "'SOMVAL', SOMVAL", "'CCD', CCD", "'SAIn', SAIn", "'PH', PH", "'CENTRO_MODERADO', CENTRO_MODERADO", "'P_LIB', P_LIB", "'CCD_CI', CCD_CI", "'UPL', UPL", "'PCOE', PCOE", "'AND', AND", "'JXC', JXC", "'PFyV', PFyV", "'CILUS', CILUS", "'PxC', PxC", "'MAS', MAS", "'IZAR', IZAR", "'UNIDAD_DEL_PUEBLO', UNIDAD_DEL_PUEBLO", "'PREPAL', PREPAL", "'Ln', Ln", "'REPO', REPO", "'INDEPENDIENTES_FIA', INDEPENDIENTES_FIA", "'ENTABAN', ENTABAN", "'IMC

In [75]:
df_votes_expl = df_votes.select('Codigo', 'Mesas', 'Censo', 'Votantes', 'Validos',
                                fn.expr("stack(" + str(len(parties)) + ", " + string_of_cols + ") as (Partido, Votos)")).\
                where(col('Votos') > 0)


In [76]:
df_votes_expl.show()

+------+-----+-----+--------+-------+--------------------+-----+
|Codigo|Mesas|Censo|Votantes|Validos|             Partido|Votos|
+------+-----+-----+--------+-------+--------------------+-----+
|  4001|    2| 1062|     823|    814|              Blanco|    5|
|  4001|    2| 1062|     823|    814|               Nulos|    9|
|  4001|    2| 1062|     823|    814|                  PP|  267|
|  4001|    2| 1062|     823|    814|                PSOE|  356|
|  4001|    2| 1062|     823|    814|     PODEMOS_IU_EQUO|   65|
|  4001|    2| 1062|     823|    814|                  Cs|  110|
|  4001|    2| 1062|     823|    814|               PACMA|    4|
|  4001|    2| 1062|     823|    814|                UPyD|    4|
|  4001|    2| 1062|     823|    814|                PCPE|    1|
|  4001|    2| 1062|     823|    814|                  EB|    2|
|  4002|    2| 1039|     748|    740|              Blanco|    2|
|  4002|    2| 1039|     748|    740|               Nulos|    8|
|  4002|    2| 1039|     

In [77]:
df_cities = spark.read.csv('PECMunicipios.csv', sep=";", header=True, encoding='ISO-8859-1')

In [78]:
df_cities = df_cities.withColumn('Comunidad', fn.trim(col('Comunidad'))).\
    withColumn('Provincia', fn.trim(col('Provincia'))).\
    withColumn('Municipio', fn.trim(col('Municipio'))).\
    withColumnRenamed('Codigo', 'CodigoPoblacion')


In [79]:
df_cities.take(1)

[Row(CodigoPoblacion='4001', Comunidad='Andalucía', Provincia='Almería', Municipio='Abla', Poblacion='1342')]

In [80]:
df_result = df_cities.join(df_votes_expl, df_cities.CodigoPoblacion == df_votes_expl.Codigo, how='inner').\
    drop('CodigoPoblacion', 'Censo', 'Votantes', 'Validos', 'Mesas')

In [81]:
df_result.printSchema()

root
 |-- Comunidad: string (nullable = true)
 |-- Provincia: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Poblacion: string (nullable = true)
 |-- Codigo: string (nullable = true)
 |-- Partido: string (nullable = true)
 |-- Votos: string (nullable = true)



In [83]:
df_result.write.csv('votes', header=True)

In [84]:
!mv votes/part*.csv votes.csv